In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

# Choose an LLM Model

In [ ]:
! pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Create your tools

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: float, b: float) -> float:
    """add two numbers."""
    return a + b


# Let's inspect some of the attributes associated with the tool.
print(add.name)
print(add.description)
print(add.args)

add
add two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [ ]:
tools_list = [add]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

# Bind tools with LLMs

In [ ]:
# we create a tool calling Agent by binding a list of tools to the llm

llm_with_tools = llm.bind_tools(tools_list)

In [ ]:

response = llm_with_tools.invoke("What is 10 plus 40?")

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'b0yrpb1rz', 'function': {'arguments': '{"a":10,"b":40}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 220, 'total_tokens': 235, 'completion_time': 0.040603258, 'prompt_time': 0.031075931, 'queue_time': 0.212526774, 'total_time': 0.071679189}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1af3c557-79b3-4e3f-aaa5-14d1fc009c71-0', tool_calls=[{'name': 'add', 'args': {'a': 10, 'b': 40}, 'id': 'b0yrpb1rz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 15, 'total_tokens': 235})

In [ ]:
# Exercise: Modify the prompt in various ways

# Tool Invokation

Tool calling is NOT tool execution

In [ ]:
response.tool_calls

[{'name': 'add',
  'args': {'a': 10, 'b': 40},
  'id': 'b0yrpb1rz',
  'type': 'tool_call'}]

In [ ]:
tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0])

ToolMessage(content='50.0', name='add', tool_call_id='b0yrpb1rz')